In [1]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup

try:
    import surprise
except ModuleNotFoundError:
    !pip install scikit-surprise
    import surprise
try:
    import scipy as sp
except ModuleNotFoundError:
    !pip install scipy
    import scipy as sp

from sklearn.metrics.pairwise import cosine_similarity
from skelarn.

In [2]:
# Import
header = {
    'User-Agent': 'Chrome 108.0.5359.125',
    'Accept-Language': 'es'
}

In [31]:
class extraccionSinopsisPeliculas():
    def __init__(self):
        self.cargarFicheroSinopsisDataframe()
    
    def obtenerPeliculasSinId(self):
        self.df_links = pd.read_csv('csv/links.csv')
        self.df_links['tmdbId'] = df_links['tmdbId'].fillna(0)
        contador = []
        for i in range(len(df_links['tmdbId'])):
            if df_links["tmdbId"][i]==0:
                contador.append(df_links["movieId"][i])
                
        return contador
    
    
    def scrapingSinopsisPeliculas(self):
        columnaSinopsis = []
        for idPeli in df_links['tmdbId']:
            print(idPeli)
            if idPeli!=0:
                try:
                    url = "https://www.themoviedb.org/movie/"+str(int(idPeli))
                    page = requests.get(url,headers=header)
                    soup = BeautifulSoup (page.content, 'html.parser')
                    sinopsis = soup.find(class_="overview")
                    sinopsis = str(sinopsis.text)
                    columnaSinopsis.append(sinopsis)
                except:
                    columnaSinopsis.append("Sin Informacion")
            else:
                columnaSinopsis.append("Sin Informacion")

    def escribirSinopsisFichero(self):
        with open(r'sinopsis2.txt', "a", encoding="utf-8") as file:
            for i in columnaSinopsis:
                raw = repr(i)
                raw_replace = raw.replace('\\n', "").replace("'","")
                file.write(raw_replace+"|||")


    def cargarFicheroSinopsisDataframe(self):
        self.df_sinopsis = pd.DataFrame(columns=["sinopsis"])
        with  open('sinopsis2.txt', 'r', encoding="utf-8") as file:
            contents = file.read()
            contents = contents.split('|||')
            
        file.close()
        self.df_sinopsis["sinopsis"] =  contents
        self.df_sinopsis = self.df_sinopsis.iloc[:-1 , :]
        return self.df_sinopsis
    
    
    
    

extract = extraccionSinopsisPeliculas()  
df_sinopsisFinal = extract.cargarFicheroSinopsisDataframe()
df_movies = pd.read_csv('csv/movies.csv')
df_moviesSinopsis = pd.concat([df_movies, df_sinopsisFinal], axis=1)
df_moviesSinopsis

,movieId,title,genres,sinopsis
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"Los juguetes de Andy, un niño de seis años, te..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,Alan Parris queda atrapado en un juego de mesa...
2,3,Grumpier Old Men (1995),Comedy|Romance,El verano ha llegado a Minnesotta y la relació...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"Basada en el best seller de Terry McMillan, la..."
4,5,Father of the Bride Part II (1995),Comedy,"Tras la boda de su querida hija Anne, George B..."
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,Susurros se convierten en titulares gritando e...
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,Es una precuela situada 6000 años antes de la ...
9739,193585,Flint (2017),Drama,Una mujer se ocupa del escándalo del agua tóxi...
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,La historia está protagonizada por una liga de...


In [37]:
def recomendacionPorSinopsis(nombrePelicula, n_similares):
    df_moviesSinopsis[df_moviesSinopsis["title"]==nombrePelicula]
    
    
recomendacionPorSinopsis("No Game No Life: Zero (2017)", 1)

,movieId,title,genres,sinopsis
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,Es una precuela situada 6000 años antes de la ...


In [2]:
class Procesos:
    def __init__(self):
        self.cargaDocumentos()
    def cargaDocumentos(self):
        self.df_links = pd.read_csv('csv/links.csv')
        self.df_links = self.df_links.dropna()
        self.df_movies = pd.read_csv('csv/movies.csv')
        self.df_movies = self.df_movies.dropna()
        self.df_ratings = pd.read_csv('csv/ratings.csv')
        self.df_ratings = self.df_ratings.dropna()
        self.df_tags = pd.read_csv('csv/tags.csv')
        self.df_tags = self.df_tags.dropna()
        self.df_movies_ratings = self.df_ratings.merge(self.df_movies)[['userId','movieId','title', 'rating','genres']]
        
        self.df_movies_ratings_tags = pd.merge(self.df_movies_ratings, self.df_tags, how='outer')[['userId','movieId','title', 'rating','genres', 'tag']]
        self.df_movies_ratings_tags["tag"] = self.df_movies_ratings_tags["tag"].str.lower()
        #self.df_movies_ratings_tags.fillna("vacio", inplace = True)
        
        self.ratings_table = self.df_movies_ratings.pivot_table(index='userId', columns='title', values='rating')
        #para cambiar los NAN por 0:
        self.ratings_table.fillna(0, inplace=True)
    def recomedacionPorValoracionOtrosUsuarios(self, nombrePelicula, n_similares):
        sparse_rating = sp.sparse.csr_matrix(self.ratings_table)
        similitud_usuarios = cosine_similarity(sparse_rating)
        #se hace con la transpuesta de la matriz creada anteriormente
        similitud_movies = cosine_similarity(sparse_rating.T)
        df_similitud_usuarios = pd.DataFrame(similitud_usuarios, index=self.ratings_table.index, columns=self.ratings_table.index)
        df_similitud_movies = pd.DataFrame(similitud_movies, index=self.ratings_table.columns, columns=self.ratings_table.columns)
        #le sumamos uno a n_similares porque la primera siempre es la propia pelicula y nos la saltamos
        n_similares+=1
        contador = 1
        print('Peliculas similares a ' + nombrePelicula + ':')
        print()
        for movie in df_similitud_movies[nombrePelicula].sort_values(ascending=False).index[1:n_similares]:
            print(str(contador) + ' - ' + str(movie))
            contador+=1
    def recomedacionPorGenero(self, nombrePelicula, n_similares):
        genres = list(set([genre for genres in self.df_movies["genres"].str.split("|") for genre in genres]))
        genre_matrix = []
        for index, row in self.df_movies.iterrows():
            genre_list = row["genres"].split("|")
            genre_vector = [1 if genre in genre_list else 0 for genre in genres]
            genre_matrix.append(genre_vector)
        genre_matrix = pd.DataFrame(genre_matrix, columns=genres)
        contador = 1
        selected_movie = self.df_movies[self.df_movies["title"] == nombrePelicula]
        selected_movie_index = selected_movie.index[0]
        #sacamos las similitudes de los generos
        similarities = cosine_similarity(genre_matrix[selected_movie_index:selected_movie_index+1], genre_matrix).flatten()
        #las metemos en una tupla y las ordenamos de mayor a menor 
        movie_list = [(index, similarity) for index, similarity in enumerate(similarities)]
        movie_list.sort(key=lambda x: x[1], reverse=True)

        print('Peliculas similares a ' + nombrePelicula + ':')
        print()
        #la bandera nos sirve para saltarnos la propia peli que buscamos
        #siempre esta a false y si nos encontramos la peli que estamos buscando la activamos a True
        #si esta en True al finalizar el bucle significa que ha saltado el titulo que buscabamos para no repetirse a si mismo 
        #y por lo tanto hay que añadir uno mas para llegar al numero deseado por el usuario
        bandera=False
        for movie in movie_list[0:n_similares]:
            if(nombrePelicula != self.df_movies.iloc[movie[0]]["title"]):
                print(str(contador)+' - ' +self.df_movies.iloc[movie[0]]["title"])
                contador+=1
            else:
                bandera=True
        if(bandera):
            #print('bandera')
            mov=movie_list[n_similares][0]
            print(str(contador)+' - ' +self.df_movies.iloc[mov]["title"])
    def predecirRatingDeUserAPeliculaPorSusGeneros(self, nombrePelicula, user_id):
        yaVotado = self.df_movies_ratings[(self.df_movies_ratings['title']==nombrePelicula) & (self.df_movies_ratings['userId']==user_id)]["rating"].unique()
        if(len(yaVotado)!=0):
            prediction = yaVotado[0]
            print()
            print("La prediccion para " + nombrePelicula+" es: " + str(prediction))
            #return prediction
        else:
            # obtener géneros de la película a predecir
            movie_genres = self.df_movies_ratings[self.df_movies_ratings['title']==nombrePelicula]["genres"].unique()
            generosPeli = movie_genres[0].split("|")
            # filtrar valoraciones del usuario para peliculas con generos en comun
            user_ratings_ID = self.df_movies_ratings[self.df_movies_ratings['userId'] == user_id]
            user_ratings = user_ratings_ID.loc[user_ratings_ID['genres'].str.split('|').apply(lambda x: any(i in x for i in generosPeli))]
            # calcular la media de valoraciones del usuario para las peliculas con generos en comun
            if user_ratings.empty:
                print("La lista es empty")
                #return None
            else:
                #prediction = user_ratings_ID['rating'].mean()
                prediction = format(user_ratings['rating'].mean(), '.3f')
                print()
                print("La prediccion para " + nombrePelicula + " es: " + str(prediction))
                #return prediction
    def recomedacionPorTags(self, nombrePelicula, n_similares):
        count_matrix = self.df_movies_ratings_tags.pivot_table(index='movieId', columns='tag', values='userId')
        #count_matrix = self.df_movies_ratings_tags.pivot_table(index='movieId', columns='tag', values='rating')
        count_matrix.fillna(0, inplace=True)
        sparse_rating = sp.sparse.csr_matrix(count_matrix)
        #print(sparse_rating)    
        selected_movie = self.df_movies[self.df_movies["title"] == nombrePelicula]["movieId"].values[0]
        #print(selected_movie)

        #encontramos el id de la pelicula en la matriz
        selected_movie_index = count_matrix.index.get_loc(selected_movie)

        similarities = cosine_similarity(sparse_rating, sparse_rating[selected_movie_index])

        movie_list = [(index, similarity) for index, similarity in enumerate(similarities)]
        movie_list.sort(key=lambda x: x[1], reverse=True)
        
        print('Peliculas similares a ' + nombrePelicula + ':')
        print()
        bandera=False
        contador = 1
        for movie in movie_list[0:n_similares]:
            if(nombrePelicula != self.df_movies.iloc[movie[0]]["title"]):
                print(str(contador)+' - ' +self.df_movies.iloc[movie[0]]["title"])
                contador+=1
            else:
                bandera=True
        if(bandera):
            mov=movie_list[n_similares][0]
            print(str(contador)+' - ' +self.df_movies.iloc[mov]["title"])
    def predecirRatingDeUserAPeliculaPorSusTags(self, nombrePelicula, user_id):
        yaVotado = self.df_movies_ratings[(self.df_movies_ratings['title']==nombrePelicula) & (self.df_movies_ratings['userId']==user_id)]["rating"].unique()
        if(len(yaVotado)!=0):
            prediction = yaVotado[0]
            print()
            print("La prediccion para " + nombrePelicula+" es: " + str(prediction))
            #return prediction
        else:
            # obtener tags de la película a predecir
            tagsPeli = []
            movie_tags = self.df_movies_ratings_tags[self.df_movies_ratings_tags['title']==nombrePelicula]["tag"].unique()
            for m in movie_tags:
                tagsPeli.append(m)
            #print(tagsPeli)
            filtroMergeandoTags = self.df_movies_ratings_tags[['userId','movieId','title', 'rating', 'tag']]
            filtroEnBaseUserId = filtroMergeandoTags[filtroMergeandoTags['userId']==1]
            user_ratings = filtroEnBaseUserId[filtroEnBaseUserId['tag'].isin(tagsPeli)]
            #si el usuario a creado un tag de alguna peli que sea igual a alguno de el de la pelicula buscada filtramos mas el df quitando los nulos
            #si no a hecho ningun tag y todos sus tag son nan dejamos el df como esta ya que si hacemos dropna eliminamos el df entero
            if user_ratings.dropna().size != 0:
                user_ratings = user_ratings.dropna()
            user_ratings
            # calcular la media de valoraciones del usuario para las peliculas con generos en comun
            if user_ratings.empty:
                print("La lista es empty")
                #return None
            else:
                #prediction = user_ratings_ID['rating'].mean()
                prediction = format(user_ratings['rating'].mean(), '.3f')
                print()
                print("La prediccion para " + nombrePelicula + " es: " + str(prediction))
                #return prediction

            

p = Procesos()

#Extra
p.recomedacionPorValoracionOtrosUsuarios("Star Wars: Episode IV - A New Hope (1977)", 10)

#Recomendaciones en base a caracteristicas sueltas
p.recomedacionPorGenero("Star Wars: Episode IV - A New Hope (1977)", 10)
p.recomedacionPorTags("Star Wars: Episode IV - A New Hope (1977)", 10)

#Prediccion de rating mediante un usuario dado
p.predecirRatingDeUserAPeliculaPorSusGeneros("Star Wars: Episode IV - A New Hope (1977)", 1)
p.predecirRatingDeUserAPeliculaPorSusTags("City of God (Cidade de Deus) (2002)", 1)

#Recomendacion en base a un usuario y caracteristicas


Peliculas similares a Star Wars: Episode IV - A New Hope (1977):

1 - Star Wars: Episode V - The Empire Strikes Back (1980)
2 - Star Wars: Episode VI - Return of the Jedi (1983)
3 - Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
4 - Matrix, The (1999)
5 - Indiana Jones and the Last Crusade (1989)
6 - Back to the Future (1985)
7 - Star Wars: Episode I - The Phantom Menace (1999)
8 - Terminator, The (1984)
9 - Godfather, The (1972)
10 - Saving Private Ryan (1998)
Peliculas similares a Star Wars: Episode IV - A New Hope (1977):

1 - Waterworld (1995)
2 - Stargate (1994)
3 - Demolition Man (1993)
4 - Star Wars: Episode V - The Empire Strikes Back (1980)
5 - Star Wars: Episode VI - Return of the Jedi (1983)
6 - Star Trek III: The Search for Spock (1984)
7 - Lost in Space (1998)
8 - Rocketeer, The (1991)
9 - Tron (1982)
10 - Six-String Samurai (1998)
Peliculas similares a Star Wars: Episode IV - A New Hope (1977):

1 - Misérables, Les (1995)
2 - Double Happine

In [3]:
prueba = Procesos()
prueba.df_movies_ratings_tags

,userId,movieId,title,rating,genres,tag
0,1,1,Toy Story (1995),4.0,Adventure|Animation|Children|Comedy|Fantasy,NaN
1,5,1,Toy Story (1995),4.0,Adventure|Animation|Children|Comedy|Fantasy,NaN
2,7,1,Toy Story (1995),4.5,Adventure|Animation|Children|Comedy|Fantasy,NaN
3,15,1,Toy Story (1995),2.5,Adventure|Animation|Children|Comedy|Fantasy,NaN
4,17,1,Toy Story (1995),4.5,Adventure|Animation|Children|Comedy|Fantasy,NaN
...,...,...,...,...,...,...
102879,573,6016,NaN,NaN,NaN,not seen
102880,573,6157,NaN,NaN,NaN,bad
102881,573,6157,NaN,NaN,NaN,ben affleck
102882,600,273,NaN,NaN,NaN,gothic
